In [1]:
import sys
sys.path.append('/home/jupyter/notebooks/src')

from utils.data_generator import Data_Generator
from utils.vocabulary import Vocabulary
from models.dense_conv import Model
import keras
import os

Using TensorFlow backend.


In [2]:
# Data Paths
train_file = '/home/jupyter/data/train.csv'
test_file = '/home/jupyter/data/test.csv'
complete_file = '/home/jupyter/data/cleaned.csv'
glove_file = '/home/jupyter/data/vocab/glove.6B.100d.txt'
max_nb_words = 200000
exp_name = 'run103'
model_dir = os.path.join('/home/jupyter/data/model_snapshots',exp_name)
log_dir = os.path.join('/home/jupyter/tf-logs/',exp_name)

os.makedirs(model_dir,exist_ok=True)
os.makedirs(log_dir,exist_ok=True)

In [3]:
vocab = Vocabulary()
vocab.setup_corpus_vocabulary(complete_file,max_nb_words)
print('Setup Vocab')
vocab.load_glove_vocabulary(glove_file)
print('Loaded glove')
vocab.construct_embedding_matrix()
print('Loaded embedding matrix')

Setup Vocab
Loaded glove
Loaded embedding matrix


In [4]:
arch = Model(vocab)
print(arch.model.summary())

/home/jupyter/notebooks/src/models/dense_conv.py:65: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  merged = Merge(mode=lambda x: tf.stack([x[0],x[1],x[2],x[3]],axis=1), output_shape=lambda x: (x[0][0],1200))([q1_dense,q2_dense,q1_conv_out,q2_conv_out])


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 100)    9101500     input_1[0][0]                    
                                                                 input_2[0][0]                    
                                                                 input_1[0][0]                    
                                                                 input_2[0][0]                    
__________

In [ ]:
train_gen = Data_Generator(vocab,32,train_file)
train_gen.load_data()
test_gen = Data_Generator(vocab,32,test_file)
test_gen.load_data()

In [ ]:
file_name = 'dense-conv'
check_cb = keras.callbacks.ModelCheckpoint(os.path.join(model_dir,file_name + '.{epoch:02d}-{val_loss:.2f}-{val_acc:.2f}.hdf5'),
                                   monitor='val_acc',
                                   verbose=0, save_best_only=True, mode='max')
tbCallback = keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=0, write_graph=True, write_images=True)
earlystop_cb = keras.callbacks.EarlyStopping(monitor='val_loss', patience=25, verbose=1, mode='auto')

arch.model.fit_generator(generator = train_gen.gen_next(), steps_per_epoch=20000, epochs=20, verbose=1
                         , callbacks=[check_cb,tbCallback]
                         , validation_data=test_gen.gen_next(), validation_steps=500, max_queue_size=10
                         , workers=3, use_multiprocessing=True, shuffle=True, initial_epoch=0)


/opt/conda/lib/python3.5/site-packages/keras/engine/training.py:2087: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


Epoch 1/20
19994/20000 [============================>.] - ETA: 0s - loss: 0.5811 - acc: 0.6945

/opt/conda/lib/python3.5/site-packages/keras/engine/training.py:2330: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


20000/20000 [==============================] - 155s 8ms/step - loss: 0.5811 - acc: 0.6945 - val_loss: 0.5511 - val_acc: 0.7341
Epoch 2/20
20000/20000 [==============================] - 152s 8ms/step - loss: 0.5367 - acc: 0.7337 - val_loss: 0.5071 - val_acc: 0.7601
Epoch 3/20
20000/20000 [==============================] - 152s 8ms/step - loss: 0.5192 - acc: 0.7467 - val_loss: 0.5093 - val_acc: 0.7494
Epoch 4/20
20000/20000 [==============================] - 152s 8ms/step - loss: 0.5063 - acc: 0.7541 - val_loss: 0.5087 - val_acc: 0.7620
Epoch 5/20
20000/20000 [==============================] - 152s 8ms/step - loss: 0.4968 - acc: 0.7610 - val_loss: 0.4934 - val_acc: 0.7696
Epoch 6/20
20000/20000 [==============================] - 152s 8ms/step - loss: 0.4894 - acc: 0.7649 - val_loss: 0.4938 - val_acc: 0.7618
Epoch 7/20
18558/20000 [==========================>...] - ETA: 10s - loss: 0.4872 - acc: 0.7670

In [ ]:
# Persist the vocabulary
import pickle
vocab_path = os.path.join(model_dir,'vocab.pickle')
with open(vocab_path, 'wb') as handle:
    pickle.dump(vocab.word_index, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
import json
model_architecture = json.loads(arch.model.to_json())

arch_path = os.path.join(model_dir,'arch.json')

json.dump(open(arch_path,'w'),model_architecture,indent=4)